In [ ]:
!pip install wget | grep -v 'already satisfied' #(added section to remove clutter of dependancies alredy installed)
!pip install PyPDF2 | grep -v 'already satisfied'
!pip install textract | grep -v 'already satisfied'
!pip install PyMuPDF | grep -v 'already satisfied'

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=402a05023b0268f15981075c0c368187da7948d7aec708e12b8c4725355c5b23
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 33.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import bs4
import wget
import fitz
import numpy as np
import PyPDF2
import textract
import re
from PyPDF2 import PdfReader
from nltk.corpus import stopwords
import string
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
import nltk
import urllib.request
from urllib.request import urlopen
from urllib.request import urlparse
from ctypes.util import find_library
import csv
from google.colab import drive
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#extracting the titel and page number
def titel(file):
    reader = PdfReader(file)
    meta = reader.metadata
    #print(len(reader.pages))
    #print(meta.title)
    return len(reader.pages)
#titel(file)

#extract the text from the pdf
def text(file):
    pdfReader = PyPDF2.PdfReader(file)
    num_pages = len(pdfReader.pages)

    count = 0
    text = ""

    while count < num_pages:
        #error pageObj = pdfReader.getPage(count)
        pageObj = pdfReader.pages[count]
        count +=1
        text += pageObj.extract_text()

    if text != "":
        text = text

    else:
        text = textract.process('words.txt', method='tesseract', language='eng')


    text = text.encode('ascii','ignore').lower()
    text_decoded = text.decode()
    return text_decoded

#Pagewise Implimentation


def extract_page(file,num=0):
    pdf_file = fitz.open(file)
    page_texts = []

    for page_number in range(len(pdf_file)):
        page = pdf_file.load_page(page_number)
        page_text = page.get_text()

        # Encode the page text as ASCII and convert to lowercase
        page_text = page_text.encode('ascii', 'ignore').lower()
        page_text_decoded = page_text.decode()
        page_texts.append(page_text_decoded)

    #page=int(input("Enter Page Number: "))
    filepage = ""

    if page_texts:
      filepage = page_texts[num]


    return filepage


In [ ]:
def cleanText(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = text.replace("/", " ")
    text = text.replace("\n", " ")
    text = text.replace("''", "")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text
round1 = lambda x: cleanText(x)

#Remove stopwords
stopwords_en = stopwords.words('english')
additionalStopWords = ["pg","figure","u"]    #Remove the words that are present in the title
stopwords_en.extend(additionalStopWords)
def toggleStopword(newStopword):
  if newStopword in stopwords_en:
        stopwords_en.remove(newStopword)
  else:
        stopwords_en.append(newStopword)

def removeStopwords(texts):
    extract = ''
    for word in simple_preprocess(texts):
        if word not in stopwords_en:
            extract = extract + ' ' + word
    return extract

#Lemmatize keywords
def lemmatizeKeywords(keywords):
    keywords_list = keywords.split(' ')
    lemmatizer = WordNetLemmatizer()
    keywords_lemmatized = []
    for keyword in keywords_list:
        keywords_lemmatized.append(lemmatizer.lemmatize(keyword))
    return ' '.join(keywords_lemmatized)


def identifyFrequentKeywords(keywordsFlat):
    keywords = keywordsFlat.split(' ')
    while '' in keywords:
        keywords.remove('')
    keywordsFlat = ' '.join(keywords)
    keywordCount = len(keywords)

    trigrams_keys = []
    trigrams_values = []
    trigrams = nltk.trigrams(keywords)
    frequency = nltk.FreqDist(trigrams)
    for key,value in frequency.items():
        if value > 1: #value->absolute count
            trigrams_keys.append(' '.join(key))
            trigrams_values.append(value/keywordCount) #->persisting relative frequency

    bigrams_keys = []
    bigrams_values = []
    bigrams = nltk.bigrams(keywordsFlat.split(' '))
    frequency = nltk.FreqDist(bigrams)
    for key,value in frequency.items():
        if value > 1:
            bigrams_keys.append(' '.join(key))
            bigrams_values.append(value/keywordCount)

    monograms_keys = []
    monograms_values = []
    frequency = nltk.FreqDist(keywordsFlat.split(' '))
    for key,value in frequency.items():
        if value > 1:
            monograms_keys.append(key)
            monograms_values.append(value/keywordCount)

    keys_selected = trigrams_keys + bigrams_keys + monograms_keys
    keys_selected_values = trigrams_values + bigrams_values + monograms_values
    keys_selected = [x for _, x in sorted(zip(keys_selected_values, keys_selected), reverse=True)]
    keys_selected_values.sort(reverse=True)

    #select top-n keywords
    numberOfTopWords = 5
    if len(keys_selected) > numberOfTopWords:
        keys_selected        = keys_selected[:numberOfTopWords]
        keys_selected_values = keys_selected_values[:numberOfTopWords]

    return keys_selected, keys_selected_values, keywordCount

#mostFrequentKeywords, relativeKeywordFrequency = identifyFrequentKeywords(lemmatizeKeywords(removeStopwords(round1(text(file)))))
#print(mostFrequentKeywords)
#print(relativeKeywordFrequency)

In [ ]:
def word_count(word, text1):
    with open('/content/drive/MyDrive/test6.txt','w+') as f:
        #f.write(cleanText(text(file1)))
        f.write(text1)
    with open('/content/drive/MyDrive/test6.txt') as f:
        return ''.join(f).count(word)


#print("Number of words in the file :", word_count(input('give me a word')))

def word_cooccurrence(word1,word2,file):
  page_list=[]
  pages=titel(file)
  for i in range(pages):
    n=extract_page(file,i)
    n=n.lower()
    if word1.lower() in n:
      if word2.lower() in n:
        page_list.append(i+1)
  return page_list


In [ ]:
def CSVFreq(file):
  pages=titel(file)
  CSV=[]
  for i in range(pages):
      n=extract_page(file,i)
      x=n
      x=cleanText(x)
      x=lemmatizeKeywords(x)                                  #Turn words into their root ( Happily to Happy, faster to fast, etc)
      x=removeStopwords(x)                                    #Only to be used if there's any specific words we want to remove
      m=identifyFrequentKeywords(x)                           #Produces two element list of lists
      a,b,c=m                                                 #alloting each list to a seperate variable
      print(i+1)
      print(m)
      for j in range(len(a)):
          CSV.append([i+1,a[j],b[j],c])
  return CSV

In [ ]:
def SetCountry(country):#Write Country name, dont forget capitals first letter
  drive.mount('/content/drive')
  file = open('/content/drive/MyDrive/Sparsh_Policies/'+country+'.pdf','rb')
  return file,country;

file,country=SetCountry("EU")
print(country)

Mounted at /content/drive
EU


In [ ]:
x=word_cooccurrence("EU","Hydrogen",file)
print(x)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 18, 20]


In [ ]:
#print(CSVFreq(file))       #TODO: Add automatic removal of words in title, create local graphs for data if possible.
print(titel(file))


x=CSVFreq(file)
with open('/content/drive/MyDrive/Sparsh_Policies/'+country+'_Frequency.csv','w+',newline='') as f:
     writer = csv.writer(f)
     writer.writerows(x)

toggleStopword(country)
toggleStopword('hydrogen') # Add Hydrogen to stopwords

x=CSVFreq(file)
with open('/content/drive/MyDrive/Sparsh_Policies/'+country+'_Frequency(without_hydrogen).csv','w+',newline='') as f:
     writer = csv.writer(f)
     writer.writerows(x)

toggleStopword('hydrogen')  # Remove Hydrogen from stopwords again for next file
toggleStopword(country)


21
1
(['european', 'final', 'en', 'council', 'committee'], [0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091, 0.09090909090909091], 22)
2
(['energy', 'supply', 'russian', 'russia', 'repowereu'], [0.06976744186046512, 0.023255813953488372, 0.023255813953488372, 0.023255813953488372, 0.023255813953488372], 172)
3
(['energy', 'eu', 'state', 'member state', 'member'], [0.034055727554179564, 0.030959752321981424, 0.021671826625386997, 0.021671826625386997, 0.021671826625386997], 323)
4
(['energy', 'saving', 'efficiency', 'measure', 'eu'], [0.09121621621621621, 0.030405405405405407, 0.02702702702702703, 0.02027027027027027, 0.016891891891891893], 296)
5
(['energy', 'gas', 'demand', 'commission', 'state'], [0.03832752613240418, 0.020905923344947737, 0.017421602787456445, 0.017421602787456445, 0.013937282229965157], 287)
6
(['eu', 'platform', 'energy', 'supply', 'state'], [0.03296703296703297, 0.03021978021978022, 0.03021978021978022, 0.013736263736263736, 0.0